In [2]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()
from Utilities.symbolicFunctions import *
from Utilities.kukaKinematics import Slist, Mlist

# Task 3
## 3.2 
### Develop and implement a solution for the analytic inverse kinematics for the Agilus robot.

In [8]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

M = Mlist[5]

#Thetas_gen can be modified to get a valid desired position T_sd
thetas_gen = np.array([1,-1,-1.3,0,2,3])
T_sd = mr.FKinSpace(M,Slist,thetas_gen)


# Finding the thetas
thetas_up, thetas_down = agilus_analytical_IK(Slist, M, T_sd)

# Resulting end effector pose and Ps for elbow up:
T_up = mr.FKinSpace(M,Slist,thetas_up)
P_up = ps_from_Tsd(T_up)

# And for elbow down:
T_down = mr.FKinSpace(M,Slist,thetas_down)
P_down = ps_from_Tsd(T_down)

print(T_sd,'\n\n', T_up, '\n\n',T_down)

[[   0.27682   -0.81052   -0.51617  -18.96356]
 [  -0.16994   -0.56999    0.80389   29.534  ]
 [  -0.94578   -0.13482   -0.29552 1072.74583]
 [   0.         0.         0.         1.     ]] 

 [[   0.27682   -0.81052   -0.51617  -60.2572 ]
 [  -0.16994   -0.56999    0.80389   93.84504]
 [  -0.94578   -0.13482   -0.29552 1049.10421]
 [   0.         0.         0.         1.     ]] 

 [[   0.27682   -0.81052   -0.51617  -60.2572 ]
 [  -0.16994   -0.56999    0.80389   93.84504]
 [  -0.94578   -0.13482   -0.29552 1049.10421]
 [   0.         0.         0.         1.     ]]


## Task 3.3
### Confirm that the solution of the analytical inverse kinematics from the previous point agrees with the solution from a numerical inverse kinematics solver.


In [4]:
#Numerical solution
thetas_num, found = mr.IKinSpace(Slist,M,T_sd,[-2,0,0,0,0,0],0.0001,0.0001)

thetas_num_p = [0,0,0,0,0,0] #the post processed solution
#Post process angles to be [-pi, pi]
for i in range(0,6):
    thetas_num_p[i] = thetas_num[i]%(2*np.pi)
    if thetas_num_p[i]>np.pi:
        thetas_num_p[i] = thetas_num_p[i]-2*np.pi

print('\nGenerated T_sd: \n', T_sd, '\nGenerating thetas:', thetas_gen, '\n\n Elbow up: ', np.round(thetas_up,3), '\n Elbow down: ', np.round(thetas_down,3), '\n Numerical: ', np.round(thetas_num_p,3), found)

print("\nElbow down: ")
print("Elbow down solution viable: ", apply_joint_lim(jointLimits, thetas_down), "\n")

print("\nElbow up: ")
print("Elbow down solution viable: ", apply_joint_lim(jointLimits, thetas_up), "\n")



Generated T_sd: 
 [[   0.27682   -0.81052   -0.51617  -18.96356]
 [  -0.16994   -0.56999    0.80389   29.534  ]
 [  -0.94578   -0.13482   -0.29552 1072.74583]
 [   0.         0.         0.         1.     ]] 
Generating thetas: [ 1.  -1.  -1.3  0.   2.   3. ] 

 Elbow up:  [-2.142 -2.132  1.539 -3.142  2.248  3.   ] 
 Elbow down:  [-2.142 -0.745 -1.372 -3.142  0.725  3.   ] 
 Numerical:  [-2.142 -2.22   1.476  0.    -2.097 -0.142] True

Elbow down: 
Elbow down solution viable:  True 


Elbow up: 
Joint number:  5 is not within the limits
Elbow down solution viable:  False 



## Task 3.4
### Using the developed analytic inverse kinematics formulation, visualize the Agilus robot in both elbow-up and elbow-down configurations for the same end-effector pose.

In [5]:
from Utilities.RobotClass import * 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [6]:
Kuka = Robot(Mlist, ['z', '-z', 'x', 'x', '-z','x']) #Initializes kuka-robot object in zero-configuration
Kuka.draw_robot()

WebVisualizer(window_uid='window_0')

In [7]:
Kuka.transform(Slist, thetas_down ) # Elbow DOWN
pUp = np.array(Kuka.joints[5].coord.get_center()) #Get coordinates of {6} in elbow down 
Kuka.draw_robot()

Kuka.transform(Slist, thetas_up ) # Elbow UP
pDown = np.array(Kuka.joints[5].coord.get_center(),dtype=float)  #Get coordinates of {6} in elbow up
Kuka.draw_robot()

print("Both elbow up and elbow down config yields the same end-effector pose: ",pUp.round(4) == pDown.round(4))

WebVisualizer(window_uid='window_1')

WebVisualizer(window_uid='window_2')

Both elbow up and elbow down config yields the same end-effector pose:  [ True  True  True]
